In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

import os
import sys
import random

from tqdm.notebook import tqdm

module_path = os.path.abspath(os.path.join('..'))
abs_path = "/content/drive/MyDrive/atml"
sys.path.append(abs_path+"/models")
sys.path.append(abs_path+"/train")
sys.path.append(abs_path+"/datasets")

from datasets import load_dsprites, CustomDSpritesDataset, train_test_random_split
from beta_vae import BetaVAEDSprites
from entanglement_metric import compute_mig

# Fix seed 
torch.manual_seed(2)
random.seed(2)
np.random.seed(2)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
class NoiseGeneratorNet(nn.Module):
    def __init__(self, max_norm=1):
        super(NoiseGeneratorNet, self).__init__()

        self.max_norm = max_norm
        self.net = nn.Sequential(
            nn.Linear(5, 1200),
            nn.ReLU(),
            nn.Linear(1200, 1200),
            nn.ReLU(),
            nn.Linear(1200, 64 * 64),
        )

    def forward(self, x):
        out = self.net(x)
        out = out.view(-1, 64, 64)
        out = torch.clamp(out, min=-self.max_norm, max=self.max_norm)

        return out

In [ ]:
def get_new_entangle_distangle_model_and_optimizer():
    model = BetaVAEDSprites()
    for param in model.encoder.parameters():
        param.requires_grad = False
    model_optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4)

    return model, model_optimizer

In [ ]:
def save_model(epoch, model, optimizer, path):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
          }, path[:-4] + f'_epoch_{epoch}' + '.pth' )

In [ ]:
def load_models_and_optimizers(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']  

    return epoch, model, optimizer

In [ ]:
def test(dataset, test_loader, device,
         noisenet, noisenet_optimizer,
         ent_model, ent_optimizer,
         dis_model, dis_optimizer):

    ent_loss_func = nn.MSELoss()
    dis_loss_func = nn.MSELoss()

    noisenet.to(device)
    ent_model.to(device)
    dis_model.to(device)

    noisenet.eval()
    ent_model.eval()
    dis_model.eval()

    ent_losses = []
    dis_losses = []
    noisenet_losses = []
    total_samples = 0 

    with torch.no_grad():
        for lt_mx in tqdm(test_loader, leave=False):
            indices = dataset.latent_to_index(lt_mx)
            lt_values = dataset.retrieve_latent_values(lt_mx.numpy())[:, 1:].to(device)

            x = dataset[indices].to(device).type(torch.float)
            noise = noisenet(lt_values.type(torch.float)).to(device)
            x_mod = x + epsilon * noise

            ent_recon, _, _ = ent_model(x)
            dis_recon, _, _ = dis_model(x)

            ent_recon = ent_recon.view(-1, 64, 64)
            dis_recon = dis_recon.view(-1, 64, 64)

            ent_recon_loss = ent_loss_func(ent_recon, x_mod)
            dis_recon_loss = dis_loss_func(dis_recon, x_mod)
            noisenet_loss = ent_recon_loss - dis_recon_loss

            ent_losses.append(ent_recon_loss * len(lt_mx))
            dis_losses.append(dis_recon_loss * len(lt_mx))
            noisenet_losses.append(noisenet_loss * len(lt_mx))
            total_samples += len(lt_mx)

        avg_ent_loss = sum(ent_losses) / total_samples
        avg_dis_loss = sum(dis_losses) / total_samples
        avg_noisenet_loss = sum(noisenet_losses) / total_samples

        print(f"Val | Ent Rec Loss: {avg_ent_loss} | Dis Rec Loss: {avg_dis_loss} | Noisenet Loss: {avg_noisenet_loss}")

In [ ]:
def train(epochs, epsilon, dataset, 
          train_loader, test_loader, load_previous, device,
          noisenet, noisenet_optimizer, noisenet_path,
          ent_model, ent_optimizer, entangled_model_path,
          dis_model, dis_optimizer, disentangled_model_path):
  
    if load_previous:
        epoch, noisenet, noisenet_optimizer = load_models_and_optimizers(noisenet, noisenet_optimizer, noisenet_path)
        _, ent_model, ent_optimizer = load_models_and_optimizers(ent_model, ent_optimizer, entangled_model_path)
        _, dis_model, dis_optimizer = load_models_and_optimizers(dis_model, dis_optimizer, disentangled_model_path)

    ent_loss_func = nn.MSELoss()
    dis_loss_func = nn.MSELoss()

    noisenet.to(device)
    ent_model.to(device)
    dis_model.to(device)
    
    noisenet.train()
    ent_model.train()
    dis_model.train()

    for i in tqdm(range(epochs)):

        ent_losses = []
        dis_losses = []
        noisenet_losses = []
        total_samples = 0 

        for lt_mx in tqdm(train_loader, leave=False):
            indices = dataset.latent_to_index(lt_mx)
            lt_values = dataset.retrieve_latent_values(lt_mx.numpy())[:, 1:].to(device)

            x = dataset[indices].to(device).type(torch.float)
            noise = noisenet(lt_values.type(torch.float)).to(device)
            x_mod = x + epsilon * noise

            ent_recon, _, _ = ent_model(x)
            dis_recon, _, _ = dis_model(x)

            ent_recon = ent_recon.view(-1, 64, 64)
            dis_recon = dis_recon.view(-1, 64, 64)

            # print(ent_recon)
            # print(dis_recon)

            ent_recon_loss = ent_loss_func(ent_recon, x_mod)
            dis_recon_loss = dis_loss_func(dis_recon, x_mod)
            noisenet_loss = ent_recon_loss - dis_recon_loss

            ent_recon_loss.backward(retain_graph=True)
            dis_recon_loss.backward(retain_graph=True)
            noisenet_loss.backward()

            ent_optimizer.step()
            ent_optimizer.zero_grad()
            
            dis_optimizer.step()
            dis_optimizer.zero_grad()
            
            noisenet_optimizer.step()
            noisenet_optimizer.zero_grad()

            ent_losses.append(ent_recon_loss * len(lt_mx))
            dis_losses.append(dis_recon_loss * len(lt_mx))
            noisenet_losses.append(noisenet_loss * len(lt_mx))
            total_samples += len(lt_mx) 

        avg_ent_loss = sum(ent_losses) / total_samples
        avg_dis_loss = sum(dis_losses) / total_samples
        avg_noisenet_loss = sum(noisenet_losses) / total_samples

        print(f"Epoch: {i+1} | Ent Rec Loss: {avg_ent_loss} | Dis Rec Loss: {avg_dis_loss} | Noisenet Loss: {avg_noisenet_loss}")

        if i % 10 == 1:
            test(dataset, test_loader, device,
                 noisenet, noisenet_optimizer,
                 ent_model, ent_optimizer, dis_model, dis_optimizer)
            save_model(epoch + i + 1, noisenet, noisenet_optimizer, noisenet_path)
            save_model(epoch + i + 1, ent_model, ent_optimizer, entangled_model_path)
            save_model(epoch + i + 1, dis_model, dis_optimizer, disentangled_model_path)
            


In [ ]:
# Define paths
dataset_path = abs_path + "/datasets/dsprites.npz"
noisenet_path = abs_path + '/experiments/trained_models/noise_net.pth'
entangled_model_path = abs_path + '/experiments/trained_models/entangled_model.pth'
disentangled_model_path = abs_path + '/experiments/trained_models/disentangled_model.pth'

In [ ]:
## Prepare dataset

dataset = CustomDSpritesDataset(load_dsprites(dataset_path, False))

latent_matrix = dataset.sample_latent(len(dataset))
# sample_latent_values = dataset.retrieve_latent_values(latent_matrix)[:, 1:]

data_train, data_test = train_test_random_split(latent_matrix, 0.8)

batch_size = 64
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=False)

In [ ]:
# Define model and optimizers
noisenet = NoiseGeneratorNet()
noisenet_optimizer = optim.Adam(noisenet.parameters(), lr=3e-4)

ent_model, ent_optimizer = get_new_entangle_distangle_model_and_optimizer()
dis_model, dis_optimizer = get_new_entangle_distangle_model_and_optimizer()

In [ ]:
# Define hyperparameters
epochs = 30
load_previous = True
epsilon = .1

In [ ]:
train(epochs, epsilon, dataset, 
      train_loader, test_loader, load_previous, device,
      noisenet, noisenet_optimizer, noisenet_path,
      ent_model, ent_optimizer, entangled_model_path,
      dis_model, dis_optimizer, disentangled_model_path)

Epoch: 1 | Ent Rec Loss: 0.010092332027852535 | Dis Rec Loss: 86.66987609863281 | Noisenet Loss: -86.65958404541016


Epoch: 2 | Ent Rec Loss: 0.008881114423274994 | Dis Rec Loss: 86.53910827636719 | Noisenet Loss: -86.53013610839844


Val | Ent Rec Loss: 0.00846796203404665 | Dis Rec Loss: 86.5679702758789 | Noisenet Loss: -86.5595474243164


Epoch: 3 | Ent Rec Loss: 0.00831396784633398 | Dis Rec Loss: 86.51904296875 | Noisenet Loss: -86.51072692871094


Epoch: 4 | Ent Rec Loss: 0.007945897988975048 | Dis Rec Loss: 86.5387191772461 | Noisenet Loss: -86.5309829711914


...

Epoch: 29 | Ent Rec Loss: 0.0065191639587283134 | Dis Rec Loss: 86.53340911865234 | Noisenet Loss: -86.52701568603516


Epoch: 30 | Ent Rec Loss: 0.006491084583103657 | Dis Rec Loss: 86.55618286132812 | Noisenet Loss: -86.5495376586914


Epoch: 31 | Ent Rec Loss: 0.006475829053670168 | Dis Rec Loss: 86.5018081665039 | Noisenet Loss: -86.49529266357422


Epoch: 32 | Ent Rec Loss: 0.006460115779191256 | Dis Rec Loss: 86.5286865234375 | Noisenet Loss: -86.52244567871094


Val | Ent Rec Loss: 0.006363532040268183 | Dis Rec Loss: 86.598388671875 | Noisenet Loss: -86.59211730957031


In [ ]:
# _, ent_model, ent_optimizer = load_models_and_optimizers(ent_model, ent_optimizer, entangled_model_path)
# for params in ent_model.decoder.parameters():
#     print(params)

In [ ]:
# _, dis_model, dis_optimizer = load_models_and_optimizers(dis_model, dis_optimizer, disentangled_model_path)
# for params in dis_model.decoder.parameters():
#     print(params)

In [4]:
bce = torch.nn.BCELoss()

In [26]:
bce(torch.Tensor([.3, .84]), torch.Tensor([.3, .85]))

tensor(0.5170)

In [6]:
torch.Tensor([1, 1])

tensor([1., 1.])

In [10]:
torch.empty(3).random_(2)

tensor([0., 1., 1.])